We will now test how quickly each ansatz converges to the equilibrium energy (that corresponding to the bond length) with each classical optimization routine, to determine which is the best ansatz-optimizer combination.

Once again, we start by importing some modules and defining the `get_qubit_op(distance)` function.

In [ ]:
from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.second_q.formats.molecule_info import MoleculeInfo
from qiskit_nature.second_q.mappers import JordanWignerMapper, QubitConverter

def get_qubit_op(d):

    h2 = MoleculeInfo(["H", "H"], 
                        [(0.0, 0.0, 0.0), (0.0, 0.0, d)], 
                        charge= 0,
                        multiplicity= 1 # 2*spin + 1,
                        )

    driver = PySCFDriver.from_molecule(h2 , basis="sto3g")
    
    problem = driver.run() 
    
    num_particles=problem.num_particles
    
    num_orbitals=int(problem.num_spatial_orbitals)
    
    hamiltonian = problem.hamiltonian
    
    second_q_op = hamiltonian.second_q_op()
    
    mapper=JordanWignerMapper()
    
    converter=QubitConverter(mapper)
    
    qubit_op=converter.convert(second_q_op)

    return qubit_op,problem,converter, num_particles, num_orbitals

We will also define the function `np_sol(problem, converter)` which yields the electronic energy through diagonalization of the Hamiltonian. This will become the target energy sought by the different routines through the optimization process.

In [ ]:
from qiskit_nature.second_q.algorithms import GroundStateEigensolver, NumPyMinimumEigensolverFactory
import numpy as np

def np_sol(problem, converter):
    
    solver_ground = NumPyMinimumEigensolverFactory()
    
    calc_gs = GroundStateEigensolver(converter, solver_ground)
    
    np_result_gs = calc_gs.solve(problem)
    
    np_electronic_energy = problem.interpret(np_result_gs).electronic_energies
        
    return np_electronic_energy


Since we chose our target energy as that at bond length,

In [ ]:
d = 0.7356

(qubit_op, problem, converter, num_particles, num_orbitals) = get_qubit_op(d)

np_electronic_energy = np_sol(problem, converter)

We will now define a callback function in which intermediate results of the optimization process will be stored, so we can track how each routine is reaching that target energy. We will input it later on the `VQE()`function.

In [ ]:
def store_intermediate_result(eval_count, parameters, mean, std):
    counts.append(eval_count)
    values.append(mean)  

At this point we can start our first loop, which for each optimizer and for an ansatz of choice (in this first case we chose UCCSD):
- stores the estimated energy in each step (`counts`) in an array called `values`
- plots the convergence process
- estimates the time used to reach the target energy.


In [ ]:
from qiskit.algorithms.optimizers import SLSQP, COBYLA, SPSA
from qiskit_nature.second_q.circuit.library import UCCSD, HartreeFock
from qiskit.algorithms.minimum_eigensolvers import VQE
from qiskit_aer.primitives import Estimator as AerEstimator

import timeit # This is used for the timing

import matplotlib.pyplot as plt  
import matplotlib.colors as mcolors
from matplotlib import pyplot

# The following configuration is optionnal but yields cleaner plots

plt.rcParams['text.usetex'] = True  
plt.rcParams['figure.dpi'] = 600
plt.rcParams['savefig.dpi'] = 600

counts = []
values = []

optimizers = ['SLSQP', 'COBYLA', 'SPSA']
estimator = AerEstimator()

for o in optimizers:
    
    counts = []
    values = []

    if o == 'SLSQP':
        
        optimizer = SLSQP(maxiter=300)
        
        color = 'gold'
        
    elif o == 'COBYLA':
        
        optimizer = COBYLA(maxiter=300)
        
        color = 'darkorange'
        
    else:
        
        optimizer = SPSA(maxiter=300)
        
        color = 'saddlebrown'

    start = timeit.default_timer()

    init_state = HartreeFock(num_orbitals,num_particles, converter)
    
    ansatz = UCCSD(num_orbitals, num_particles, converter, initial_state = init_state) 
    
    vqe = VQE(estimator, ansatz, optimizer, callback = store_intermediate_result)

    result = vqe.compute_minimum_eigenvalue(operator=qubit_op)

    stop = timeit.default_timer()

    execution_time = stop - start

    plt.plot(counts, values, color = color, linestyle = '-', linewidth=1, label= o)

    print(f"{o}", f"took {execution_time: .2f}", f"seconds and {counts[-1]: .1f} counts" ,"to reach convergence", sep=' ') # It returns time in seconds
  


print('SIDENOTE: Energy calculated is ONLY electronic energy')   
plt.axhline(y = np_electronic_energy , color = 'darkgrey', linestyle = '-', linewidth=1, label='Target energy') # Thee +0.755.. is the nuclear repulsion energy computed by numpy
# pyplot.xscale('log')
plt.xlabel('Cost function evaluation count')
plt.ylabel('Energy (Hartree)')
plt.legend()
plt.savefig('uccsd_convergence.jpg') 
plt.show()

The same thing can be done with `EfficientSU2`as the ansatz of choice by switching `ansatz = UCCSD(num_orbitals, num_particles, converter, initial_state = init_state)` for `ansatz = EfficientSU2(num_qubits = 4, entanglement = 'linear', initial_state = init_state)`.
